# Sensitivity Analysis for the Lockdown multilevel model

Through the feature selection process, we have created a series of models to describe the soundscape perception. These have ended up with around 5 individual level variables defining the model for each perceptual attribute. What I'd like to find out is which of these parameters contributes the most to the uncertainty of the model. For this stage, we are disregarding continuous variables, so we should be able to run all of this with SALib within python.

## Load in the data and models

In [2]:
import os
sys.path.append("C:\\Users\\Andrew\\OneDrive - University College London\\_PhD\\Papers - Drafts\\J5_JASA_Lockdown-SS")

from scripts import lockdown_mlm as mlm
import pandas as pd
from pathlib import Path
import numpy as np
import time
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Define some constants and options
## variables
dep_vars = ["Natural", "Traffic", "Human", "Other", "loudness", "overall", "Pleasant", "Eventful"]

FEATS_LISTS = mlm.FEATS_LISTS
remove = ["FS_TEMP", "LAeq_TEMP", "LCeq_TEMP", "LZeq_TEMP", "I_TEMP", "N_TEMP", "R_TEMP", "S_TEMP", "SIL_TEMP", "THD_TEMP", "T_TEMP"]

for k in remove:
    FEATS_LISTS.pop(k, None)

acoustic_vars = sorted({x for v in FEATS_LISTS.values() for x in v})

## processing options
nonlinear_transformations = [] # Leave a blank list to do no transformations
criterion = "aic"

# ##################################################################
# Load Data

DATA_DIR = Path("C:/Users/Andrew/OneDrive - University College London/_PhD/Papers - Drafts/J5_JASA_Lockdown-SS/data")
RESULTS_DIR = Path("C:/Users/Andrew/OneDrive - University College London/_PhD/Papers - Drafts/J5_JASA_Lockdown-SS/results")
ssidData = pd.read_csv(DATA_DIR.joinpath("2020-08-13/LondonVeniceBINResults_2020-08-13_4.csv"))

for col_name in ["Lockdown"]:
    ssidData[col_name] = ssidData[col_name].astype('category')

# Cutdown the dataset
cols = ["GroupID", "LocationID", "SessionID", "Lockdown"] + dep_vars + acoustic_vars
ssidData = ssidData[cols]

# Compress to mean of each GroupID
# compressData = ssidData.copy()
compressData = ssidData.groupby(["GroupID"]).mean()
compressData = compressData.merge(ssidData[["GroupID", "LocationID", "SessionID", "Lockdown"]].drop_duplicates(),  on="GroupID")

location_codes = pd.Categorical(compressData["LocationID"]).codes
compressData["LocationID_codes"] = location_codes
compressData.loc[compressData["Lockdown"] == 1].dropna(inplace=True)
compressData = compressData.dropna(subset=acoustic_vars)

compressData, acoustic_vars = mlm.nonlinear_features(compressData, acoustic_vars, transformations=nonlinear_transformations)
print("Initial features number:    ", len(acoustic_vars))

# Standardise
from sklearn.preprocessing import StandardScaler
compressData = compressData.replace([np.inf, -np.inf], np.nan)
compressData = compressData.dropna(subset=acoustic_vars)
scaler = StandardScaler()
compressData[acoustic_vars] = scaler.fit_transform(compressData[acoustic_vars])


# ###############################################################
# Split Prelockdown from during lockdown
prelockdownData = compressData.loc[compressData["Lockdown"] == 1]
prelockdownData = prelockdownData.dropna()
print("prelockdownData shape:     ", prelockdownData.shape)
lockdownData = compressData.loc[compressData["Lockdown"] == 2]
print("during lockdownData shape: ", lockdownData.shape)


Initial features number:     119
prelockdownData shape:      (661, 132)
during lockdownData shape:  (573, 132)


In [9]:
pleasant_model = sm.load_pickle(str(RESULTS_DIR.joinpath("2020-09-01/Pleasant_rirs_model_1.pickle")))

print("Pleasant params:\n",pleasant_model.fe_params)

eventful_model = sm.load_pickle(str(RESULTS_DIR.joinpath("2020-09-01/Eventful_rirs_model_1.pickle")))

print("\nEventful params:\n",eventful_model.fe_params)

Pleasant params:
 Intercept               0.271342
LZeq                   -0.097666
THD_95                  0.018068
PeakSpectralCentroid   -0.038564
LZeq_10_LZeq_90         0.017302
I                       0.006066
dtype: float64

Eventful params:
 Intercept    0.122624
R_50         0.066210
LAeq_Min     0.041487
dtype: float64


## Import SALib

In [14]:
from SALib.sample import saltelli
from SALib.analyze import sobol

## Define the testing Model Inputs
Next, we must define the model inputs for the SA. Our pleasant model has 5 parameters (`LZeq`, `THD_95`, `PeakSpectralCentroid`, `LZeq_10_LZeq_90`, `I`). For each of these, we need to let the saltelli sampling function know what the range of values is, so we'll pull that from the original dataset. Note, these values will be z-scaled so won't look necessarily correct.

In [15]:
pleasant_problem = {
    'num_vars': 5,
    'names': ['LZeq', 'THD_95', 'PeakSpectralCentroid', 'LZeq_10_LZeq_90', 'I'],
    'bounds': [
        [prelockdownData.LZeq.min(), prelockdownData.LZeq.max()],
        [prelockdownData.THD_95.min(), prelockdownData.THD_95.max()],
        [prelockdownData.PeakSpectralCentroid.min(), prelockdownData.PeakSpectralCentroid.max()],
        [prelockdownData.LZeq_10_LZeq_90.min(), prelockdownData.LZeq_10_LZeq_90.max()],
        [prelockdownData.I.min(), prelockdownData.I.max()]
    ]
}

## Generate Samples
Next, we generate the samples. Since we are performing a Sobol sensitivity analysis, we need to generate samples using the Saltelli sampler, as shown below.

In [16]:
pleasant_param_values = saltelli.sample(pleasant_problem, 1000)

## Run Model
SALib is not involved in the evaluation of the mathematical or computational model. If the model is written in Python, then generally you will loop over each sample input and evaluate the model:

In [33]:
Y = pleasant_model.predict(pd.DataFrame(pleasant_param_values, columns=pleasant_problem['names'])).values

## Perform Analysis
With the model outputs calculated, we can finally compute the sensitivity indices. In this example, we use `sobol.analyze`, which will compute first, second, and total-order indices. 

In [34]:
Si = sobol.analyze(pleasant_problem, Y)

`Si` is a Python `dict` with the keys `"S1", "S2", "ST", "S1_conf", "S2_conf"` and `"ST_conf"`. The `_conf` keys store the corresponding confidence intervals, typically with a confidence level of 95%. Use the keyword argument `print_to_console = True` to print all indices. Or, we can print the individualv values from `Si` as shown below. 

In [38]:
print(pleasant_problem['names'])
print(Si['S1'])

['LZeq', 'THD_95', 'PeakSpectralCentroid', 'LZeq_10_LZeq_90', 'I']
[0.89729442 0.02616715 0.05962125 0.01426697 0.00906382]


In [36]:
print(Si['ST'])

[0.89654887 0.02717511 0.05956613 0.01438075 0.00929382]


## Repeat for Eventful model

In [39]:
eventful_problem = {
    'num_vars': 2,
    'names': ['R_50', 'LAeq_Min'],
    'bounds': [
        [prelockdownData.R_50.min(), prelockdownData.R_50.max()],
        [prelockdownData.LAeq_Min.min(), prelockdownData.LAeq_Min.max()],
    ]
}
eventful_param_values = saltelli.sample(eventful_problem, 1000)
Y = eventful_model.predict(pd.DataFrame(eventful_param_values, columns=eventful_problem['names'])).values

Si = sobol.analyze(eventful_problem, Y)

print(eventful_problem['names'])
print(Si['S1'])

['R_50', 'LAeq_Min']
[0.88638975 0.11611957]


In [41]:
print(Si['S2'])

[[        nan -0.00283246]
 [        nan         nan]]
